<a href="https://colab.research.google.com/github/RyanZha0/CIENE-4012/blob/main/Mobility%20data%20filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import shapely.wkt
import numpy as np
from shapely.geometry import Point, Polygon
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define Manhattan's geographic boundaries with a simplified polygon approximation.
manhattan_polygon_points = [
    (-73.934580, 40.880912),
    (-73.951977, 40.851736),
    (-74.013391, 40.754845),
    (-74.024283, 40.698188),
    (-74.009793, 40.696703),
    (-73.996477, 40.705462),
    (-73.973175, 40.708431),
    (-73.965734, 40.730099),
    (-73.965358, 40.743476),
    (-73.939437, 40.775202),
    (-73.924349, 40.798349),
    (-73.933169, 40.809234),
    (-73.933984, 40.833898),
    (-73.907016, 40.874670)
]
manhattan_polygon = Polygon(manhattan_polygon_points)

# Define Central Park's geographic boundaries
central_park_polygon_points = [
    (-73.9580, 40.8007),
    (-73.9493, 40.7968),
    (-73.9734, 40.7648),
    (-73.9817, 40.7681)
]
central_park_polygon = Polygon(central_park_polygon_points)

# Function to determine if a given point is within the specified polygon
def is_in_polygon(lng, lat, polygon):
    point = Point(lng, lat)
    return polygon.contains(point)

# Load the dataset
file_path = '/content/drive/My Drive/CIEN 4012 Project/Codes/mobility data - during covid/data/daily_ct2ct_2020_06_05.csv'  # Adjust the file path accordingly
df = pd.read_csv(file_path)

# Filter for records where the origin is within Manhattan and the destination is within Central Park
filtered_data = df[
    df.apply(lambda x: is_in_polygon(x['lng_o'], x['lat_o'], manhattan_polygon) and
             is_in_polygon(x['lng_d'], x['lat_d'], central_park_polygon), axis=1)
]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
filtered_data.to_csv('filtered_data_origin_manhattan_destination_central_park_06_05.csv')

In [ ]:
# Load datasets
file_path = 'filtered_data_origin_manhattan_destination_central_park_06_05.csv'
zip_code_file_path = 'Modified_Zip_Code_Tabulation_Areas__MODZCTA_.csv'

data = pd.read_csv(file_path)
zip_codes_data = pd.read_csv(zip_code_file_path)

# Convert the_geom column to Shapely Polygon objects in zip_codes_data
zip_codes_data['polygon'] = zip_codes_data['the_geom'].apply(lambda wkt_str: shapely.wkt.loads(wkt_str))

# Initialize a column for zip codes in the original data
data['zipcode'] = np.nan

# Match zip codes to the starting points
for idx, row in data.iterrows():
    point = Point(row['lng_o'], row['lat_o'])
    for _, zip_row in zip_codes_data.iterrows():
        polygon = zip_row['polygon']
        if polygon.contains(point):
            data.at[idx, 'zipcode'] = zip_row['MODZCTA']
            break

In [ ]:
output_file_path = 'updated_data_with_zipcodes_06_05.csv'
data.to_csv(output_file_path, index=False)
data

,Unnamed: 0,geoid_o,geoid_d,lng_o,lat_o,lng_d,lat_d,date,visitor_flows,pop_flows,zipcode
0,55501,36061006500,36061014300,-73.999577,40.729968,-73.965554,40.782477,2020-06-05,1,49.0,10012.0
1,626086,36061011500,36061014300,-73.993549,40.757168,-73.965554,40.782477,2020-06-05,2,35.0,10018.0
2,905876,36061016400,36061014300,-73.944474,40.787714,-73.965554,40.782477,2020-06-05,7,175.0,10029.0
3,1026369,36061011800,36061014300,-73.960855,40.765275,-73.965554,40.782477,2020-06-05,3,77.0,10065.0
4,1026511,36061012100,36061014300,-73.991700,40.759709,-73.965554,40.782477,2020-06-05,6,178.0,10036.0
...,...,...,...,...,...,...,...,...,...,...,...
110,8311059,36061012900,36061014300,-74.000034,40.764908,-73.965554,40.782477,2020-06-05,6,120.0,NaN
111,8434756,36061015300,36061014300,-73.980666,40.774858,-73.965554,40.782477,2020-06-05,1,76.0,10023.0
112,8577860,36061017000,36061014300,-73.942147,40.790902,-73.965554,40.782477,2020-06-05,5,121.0,10029.0
113,8956171,36061021600,36061014300,-73.954388,40.800677,-73.965554,40.782477,2020-06-05,7,203.0,10026.0


In [ ]:
# Drop rows with NA in the 'zipcode' column
data = data.dropna(subset=['zipcode'])

# Optional: Reset the index after dropping rows
data = data.reset_index(drop=True)

data['zipcode'] = data['zipcode'].astype(int)
output_file_path = 'updated_data_with_zipcodes_06_05.csv'
data.to_csv(output_file_path, index=False)

data

,Unnamed: 0,geoid_o,geoid_d,lng_o,lat_o,lng_d,lat_d,date,visitor_flows,pop_flows,zipcode
0,55501,36061006500,36061014300,-73.999577,40.729968,-73.965554,40.782477,2020-06-05,1,49.0,10012
1,626086,36061011500,36061014300,-73.993549,40.757168,-73.965554,40.782477,2020-06-05,2,35.0,10018
2,905876,36061016400,36061014300,-73.944474,40.787714,-73.965554,40.782477,2020-06-05,7,175.0,10029
3,1026369,36061011800,36061014300,-73.960855,40.765275,-73.965554,40.782477,2020-06-05,3,77.0,10065
4,1026511,36061012100,36061014300,-73.991700,40.759709,-73.965554,40.782477,2020-06-05,6,178.0,10036
...,...,...,...,...,...,...,...,...,...,...,...
102,7596388,36061031100,36061014300,-73.929519,40.846393,-73.965554,40.782477,2020-06-05,1,0.0,10033
103,8434756,36061015300,36061014300,-73.980666,40.774858,-73.965554,40.782477,2020-06-05,1,76.0,10023
104,8577860,36061017000,36061014300,-73.942147,40.790902,-73.965554,40.782477,2020-06-05,5,121.0,10029
105,8956171,36061021600,36061014300,-73.954388,40.800677,-73.965554,40.782477,2020-06-05,7,203.0,10026


In [ ]:
# Group the data by zipcode and sum the visitor_flows and pop_flows for each group
grouped_data = data.groupby('zipcode').agg({
    'visitor_flows': 'sum',
    'pop_flows': 'sum'
}).reset_index()

output_file_path = '/content/drive/My Drive/CIEN 4012 Project/Codes/mobility data - during covid/data/grouped/grouped_data_2020_06_05.csv'
grouped_data.to_csv(output_file_path, index=False)

grouped_data

,zipcode,visitor_flows,pop_flows
0,10001,1,16.0
1,10002,1,46.0
2,10003,3,112.0
3,10007,16,13.0
4,10010,2,101.0
5,10011,1,60.0
6,10012,1,49.0
7,10014,3,211.0
8,10017,1,1.0
9,10018,8,123.0
